<h1>Project J: Text Classification</h1>
<sub>Classifies text in Jerma highlights compared to the VODs<sub>

In [ ]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tensorflow-text
!pip install pandas
!pip install scikit-learn

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd
df= pd.read_csv("jerma.csv")

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df["Message"],df["content"], test_size=0.2, stratify=df['content'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string,name="input")
preprocessed_text = bert_preprocess(sentences)
outputs = bert_encoder(preprocessed_text)
# dropout layer to mitigate over fitting
l1 = tf.keras.layers.Dropout(0.1,name="dropout")(outputs["pooled_output"])
l2 = tf.keras.layers.Dense(1,activation="sigmoid",name="output")(l1)

model = tf.keras.Model(inputs=[text_input],output=[l2])

model.compile(
    optomizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(X_train, Y_train, epochs=10)
model.evaluate(X_test, Y_test)

model.save('jerma_model')
export_dir = './jerma_model'
tf.saved_model.save(model, export_dir=export_dir)